In [1]:
#1. Install Required Libraries
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-d68989ix
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-d68989ix
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=1404b20c178cdb88edf9827b141959a9e1327cc1061404da0f4932b6b48bba03
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ai4a63m/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything


In [2]:
#2. Clone the SAM Repository and Download a Model
!git clone https://github.com/facebookresearch/segment-anything.git
%cd segment-anything

# Download a pre-trained model (ViT-B, the base one is sufficient for most uses)
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

Cloning into 'segment-anything'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 304 (delta 2), reused 1 (delta 1), pack-reused 299 (from 2)
Receiving objects: 100% (304/304), 18.31 MiB | 36.98 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/segment-anything
--2025-06-11 21:31:37--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.126, 18.238.176.44, 18.238.176.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M   166MB/s    in 2.1s    

2025-06-11 21:31:40 (166 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [3]:
#3. Upload Your Images to Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
import cv2
import os
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# -- Paths --
sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"
device = "cuda" if torch.cuda.is_available() else "cpu"

input_folder = '/content/drive/MyDrive/MBL 559E - Group Project/Fifth Test/U2Net_Cl_resized_augm'           # <-- EDIT THIS
output_folder = '/content/drive/MyDrive/MBL 559E - Group Project/Fifth Test/U2Net_Cl_resized_segmn_2'    # <-- EDIT THIS

os.makedirs(output_folder, exist_ok=True)

# -- Load Model --
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
mask_generator = SamAutomaticMaskGenerator(sam)

# -- Batch Processing --
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, filename)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        masks = mask_generator.generate(image)

        if len(masks) > 0:
            # Get largest mask (by area)
            largest_mask = max(masks, key=lambda x: x['area'])
            mask = largest_mask['segmentation'].astype('uint8') * 255

            # Save mask with same base name
            mask_filename = filename.rsplit('.', 1)[0] + '_mask.png'
            mask_path = os.path.join(output_folder, mask_filename)
            cv2.imwrite(mask_path, mask)
        else:
            print(f"No mask found for {filename}")


No mask found for y_171_04_aug0.jpg
No mask found for y_009_05_aug3.jpg
No mask found for y_046_04_aug1.jpg
No mask found for y_083_02_aug1.jpg
No mask found for y_092_01_aug3.jpg
No mask found for y_102_06_aug3.jpg
